In [2]:
#!pip install sklearn
#!pip install transformers
#!pip install -U ray
#!pip install -U ray[tune]
#!pip install datasets
#!pip install seaborn

In [1]:
import numpy as np
import pandas as pd

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers import Trainer, TrainingArguments

from ray import tune
from ray.tune import CLIReporter
from ray.tune.suggest.bayesopt import BayesOptSearch

from datasets import load_metric

from cf_matrix import make_confusion_matrix

from IPython.display import clear_output
import logging
logging.basicConfig(level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore")

## Load and Split Data

In [2]:
# remove emoji from data
def no_emoji(X):
    for i in range(len(X)):
        s = ''
        count = 0
        for j in range(len(X[i])):
            if X[i][j] == "[":
                count += 1
            elif count == 0:
                s += X[i][j]
            if X[i][j] == "]" and count > 0:
                count -= 1
        X[i] = s
        
    return X


# split data to train, validation, test
def split(df, need_emoji = True, random_state = 0):
    X = list(df['review'])
    y = list(df['label'])
    
    # 60% train, 20% development, 20% test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = random_state)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = random_state)
    if not need_emoji:
        X_train = no_emoji(X_train)
        X_val = no_emoji(X_val)
        X_test = no_emoji(X_test)
    return X_train, X_val, X_test, y_train, y_val, y_test

### weibo

In [3]:
# load weibo data
df_weibo = pd.read_csv('data/processed_weibo_data.csv')
df_weibo = df_weibo.dropna().drop("Unnamed: 0", axis = 1)

In [4]:
# split weibo data to train, validation, test
# data with emoji
X_train_wb_1, X_val_wb_1, X_test_wb_1, y_train_wb_1, y_val_wb_1, y_test_wb_1 = split(df_weibo)
# same data with emoji removed
X_train_wb_0, X_val_wb_0, X_test_wb_0, y_train_wb_0, y_val_wb_0, y_test_wb_0 = split(df_weibo, need_emoji = False)

In [5]:
[X_train_wb_1[0:5], X_train_wb_0[0:5]]

[['猛然发现，「财经郎眼」之前的广告是修正牌消糜栓。[衰]',
  '看爪就知道是美女！[亲亲][爱你]',
  '你现在是在去怀柔的路上吗？[哈哈][哈哈] ',
  '时间过得太快，老感觉不够用！[泪]',
  '吃饱喝足谁也不服[哈哈][哈哈][哈哈]'],
 ['猛然发现，「财经郎眼」之前的广告是修正牌消糜栓。',
  '看爪就知道是美女！',
  '你现在是在去怀柔的路上吗？ ',
  '时间过得太快，老感觉不够用！',
  '吃饱喝足谁也不服']]

### dianping

In [6]:
# load dianping data
df_dianping = pd.read_csv('data/processed_dianping_data.csv')
df_dianping = df_dianping.dropna().drop("Unnamed: 0", axis = 1)

In [7]:
# split dianping data to train, validation, test
# dianping data does not have emoji
X_train_dp_0, X_val_dp_0, X_test_dp_0, y_train_dp_0, y_val_dp_0, y_test_dp_0 = split(df_dianping)

In [8]:
X_train_dp_0[0:5]

['中午在旺角逛街，12点多就去了附近的潮楼。那里应该算是早茶的地方的，但是中午人依然很多。还好到的早，不怎么需要等位的。领位的服务生，忙是忙，一时忽略了我们，还打招呼，不好意思。\n绫鱼丸，新鲜鱼肉做的，能吃的出有鱼刺的成分，但是绝对不会卡到的那种，很鲜；\n皮蛋瘦肉粥，好浓的，以至于我和V都说，要么回家以后也把粥煮成这样，但是味道肯定调不好的；\n虾饺，里面真的有3个虾仁，很Q的，要不是吃不下，肯定还会再点的；\n凤爪，比较酥烂，食材本身研制的比较入味，所以很好吃，V说比较适合她妈吃；\n肠粉，里面的料很好，也有虾仁的成分面还有新鲜蔬菜，结合的好香；\nXO酱萝卜糕，因为便宜就点来吃吃，绝对超过了萝卜的口感，超级好吃；\n牛肉丸，做的比较嫩，但是确很Q，跟芹菜结合的，有种特殊的香味。\n最后结帐$123，出乎意料，本来还以为要超200了。这顿真的吃的很舒服。\n',
 '鸟照烧 大家都懂了对伐 20块钱的中饭 还是蛮灵的\n就是感觉很 穷人的感觉，几片腌萝卜 还有鸟照烧 。\n米饭还是蛮好吃的\xa0\xa0MISO有点咸。\n店很古老了....\n',
 '也是从网上看到这个地方的推荐，找了半天都没有找到。瓷器口进去大概是个T型号的路，从最下面进去，往T的丁字口往右走就是江边。快到丁字口的路上，在右边有个非常窄的巷子，不注意就错过。进去才发现别有洞天，里面位置挺宽敞的，里面有个小牌子说是CCTV2推荐的。我们专程点了毛血旺，网上人说这个很地道。这边的毛血旺的确是最老方式的毛血旺，不过味道不太好，感觉重庆城里随便一个摊位上的都比这好吃。还好，点了一个折耳根还下饭，要不然真不知道吃什么。建议，没有什么必要就不要找这个地方吧。\n',
 '一直很怀念那儿的鸡，不知道是怎么做的，但是真的很嫩，每次去人基本上都客满的，还有各种炒菜，只是店小了点，值得一试\n',
 '南非世界杯，2点钟德国的半决赛是在这里看的。\n我觉得麦当劳比肯德基好，最近几年扩张后劲十足，反观肯德基似乎孱弱并且诟病不断，推出不少纯噱头难吃的要死的骗钱食品。\n加之麦咖啡进驻，麦当劳让越来越多人喜欢。\n']

## Load Models

In [23]:
tokenizer = AutoTokenizer.from_pretrained('uer/chinese_roberta_L-12_H-768')

In [26]:
# model with weibo emoji
roberta_weibo_emoji = AutoModelForSequenceClassification.from_pretrained('trained_model/roberta_weibo_emoji')
# classifier
weibo_emoji_classifier = pipeline('sentiment-analysis', model = roberta_weibo_emoji, tokenizer = tokenizer, device = 0)

In [27]:
# model without weibo emoji
roberta_weibo_no_emoji = AutoModelForSequenceClassification.from_pretrained('trained_model/roberta_weibo_no_emoji')
# classifier
weibo_no_emoji_classifier = pipeline('sentiment-analysis', model = roberta_weibo_no_emoji, tokenizer = tokenizer, device = 0)

In [28]:
# uer dianping model
uer_roberta_dianping = AutoModelForSequenceClassification.from_pretrained('uer/roberta-base-finetuned-dianping-chinese')
# classifier
uer_dianping_classifier = pipeline('sentiment-analysis', model = uer_roberta_dianping, tokenizer = tokenizer, device = 0)

## Assess Performance

In [29]:
# assess performance of model, and print a confusion matrix
def performance(X_test, y_test, classifier, threshold = 0.5):
    #convert = {'positive (stars 4 and 5)': 1, 'negative (stars 1, 2 and 3)': 0}
    convert = {'LABEL_1': 1, 'LABEL_0': 0}
    y_pred = []
    
    for review in X_test:
        prediction = classifier(review)[0]
        label = convert[prediction['label']]
        if label == 1 and prediction['score'] < threshold:
            label = 0
        y_pred.append(label)
        clear_output(wait = True)
        print("{}/{}".format(len(y_pred), len(y_test)))

    cf_matrix = confusion_matrix(y_test, y_pred)
    labels = ['TN', 'FP', 'FN', 'TP']
    categories = ['Negative', 'Positive']
    
    make_confusion_matrix(cf_matrix, group_names = labels, categories = categories, cmap = 'binary')

### weibo emoji model

In [ ]:
# emoji model performance on weibo data with emoji
performance(X_test_wb_1, y_test_wb_1, weibo_emoji_classifier)

In [ ]:
# emoji model performance on weibo data with emoji removed
performance(X_test_wb_0, y_test_wb_0, weibo_emoji_classifier)

In [ ]:
# emoji model performance on dianping data
performance(X_test_dp_0, y_test_dp_0, weibo_emoji_classifier)

### weibo no emoji model

In [ ]:
# non emoji model performance on weibo data with emoji
performance(X_test_wb_1, y_test_wb_1, weibo_no_emoji_classifier)

In [ ]:
# non emoji model performance on weibo data with emoji removed
performance(X_test_wb_0, y_test_wb_0, weibo_no_emoji_classifier)

In [ ]:
# non emoji model performance on dianping data
performance(X_test_dp_0, y_test_dp_0, weibo_no_emoji_classifier)

### dianping model

In [ ]:
# assess performance of model, and print a confusion matrix
def performance(X_test, y_test, classifier, threshold = 0.5):
    convert = {'positive (stars 4 and 5)': 1, 'negative (stars 1, 2 and 3)': 0}
    #convert = {'LABEL_1': 1, 'LABEL_0': 0}
    y_pred = []
    
    for review in X_test:
        prediction = classifier(review)[0]
        label = convert[prediction['label']]
        if label == 1 and prediction['score'] < threshold:
            label = 0
        y_pred.append(label)
        clear_output(wait = True)
        print("{}/{}".format(len(y_pred), len(y_test)))

    cf_matrix = confusion_matrix(y_test, y_pred)
    labels = ['TN', 'FP', 'FN', 'TP']
    categories = ['Negative', 'Positive']
    
    make_confusion_matrix(cf_matrix, group_names = labels, categories = categories, cmap = 'binary')

In [ ]:
# dianping model performance on weibo data with emoji
performance(X_test_wb_1, y_test_wb_1, uer_dianping_classifier)

In [ ]:
# dianping model performance on weibo data with emoji removed
performance(X_test_wb_0, y_test_wb_0, uer_dianping_classifier)

In [ ]:
# dianping model performance on dianping data
performance(X_test_dp_0, y_test_dp_0, uer_dianping_classifier)

## Investigate Misclassified Data

In [142]:
def metrics_idx(data, target, classifier):
    threshold = 0.5
    convert = {'LABEL_1': 1, 'LABEL_0': 0}
    y_pred = []

    for review in data:
        prediction = classifier(review)[0]
        label = convert[prediction['label']]
        if label == 1 and prediction['score'] < threshold:
            label = 0
        y_pred.append(label)
    
    tp = (np.array(y_pred) + np.array(target)) == 2
    tn = (np.array(y_pred) + np.array(target)) == 0
    fp = (np.array(y_pred) + 2 * np.array(target)) == 1
    fn = (np.array(y_pred) + 2 * np.array(target)) == 2

    tp_idx = np.where(tp == True)[0]
    tn_idx = np.where(tn == True)[0]
    fp_idx = np.where(fp == True)[0]
    fn_idx = np.where(fn == True)[0]
    
    return tp_idx, tn_idx, fp_idx, fn_idx

In [180]:
# weibo review with emoji on weibo emoji model
y_test_wb_1_emoji_metrics = metrics_idx(X_test_wb_1, y_test_wb_1, weibo_emoji_classifier)

In [181]:
# weibo review without emoji on weibo emoji model
y_test_wb_0_emoji_metrics = metrics_idx(X_test_wb_0, y_test_wb_0, weibo_emoji_classifier)

In [179]:
wb_tp_emoji_diff = np.setdiff1d(y_test_wb_1_emoji_metrics[0], y_test_wb_0_emoji_metrics[0])
np.array(X_test_wb_1)[wb_tp_emoji_diff][100:200]

array(['【云百科】小亭温馨提示：经常熬夜的人一定注意通过食物调节身体哦[可爱] 1．猕猴桃、猪脚可以呵护皮肤；2．绿茶、枸杞能够养肝明目； 3．鸡蛋、燕麦帮助补充大脑营养； 4．香菇、干红葡萄酒消除腹部脂肪堆积； 5．山药、杜仲可以滋养肾气；6．苹果、冬瓜消肿利尿；三文鱼、酸奶防止肌肉萎缩。#云百科#',
       '一句话证明我在上海：快下雪了，冷死了[嘻嘻] ', '[馋嘴][馋嘴][馋嘴][馋嘴]', '[偷笑]', '[鼓掌] ',
       '成南高速已经限行，整个高速如同停车场。早上回去吃米粉无望。[嘻嘻]', '[亲亲]', '[哈哈] ',
       '让您的孩子不再害怕打针[鼓掌]', '[哈哈]哥们儿下次炒作前最好能花点钱涨些粉先', '[鼓掌] ', '[闭嘴]',
       '[good] ', '[ok][ok]', '很明?，??爹?我的那?可?[嘻嘻]',
       '我特别奇怪，每次准备回父母家，大师就说，注意家庭矛盾。要不要这么准啊[哈哈] ', '明明是…………七喜[哈哈]',
       '虽然怕冷，但我还是愿意去感受北欧国家的童话感觉。[爱你]', '[哈哈] ',
       '珍惜时间模范小姐。#终于找到早高峰大堵车的大段时间该做什么啦# [爱你][嘻嘻][嘻嘻][嘻嘻][偷笑][偷笑][偷笑][偷笑]',
       '关云长语言很搞笑，很现代；情节很空洞，很简单；动作很给油，很利落。整个感觉很有让子弹飞的感觉。导演很给油，关云长竟然还有三角恋，太不厚道了！[嘻嘻]',
       '[哈哈] ',
       '不会吧？[吃惊]俺刚刚从餐厅回来，没见到主任啊！[嘻嘻]雪倒是下得很欢喜的样子。[雪][雪][雪][雪人][雪人][雪人][手套][围脖][温暖帽子] ',
       '多少天津话能评价这种行为，且一针见血[嘻嘻]。', '雪，怕是凝不住。[嘻嘻]', '这张照片真是绝了~~各种表情~~[哈哈]',
       '[哈哈][哈哈]吃过的有木有？？', '[哈哈]', '太过奢华了，只能看看啊，买是买不起[哈哈]',
       '哇，实在是发自内心的幸福啊[可爱]', '[鼓掌] ',
       '@现代可汗 @鹏的莹 追杀的可能性[嘻嘻][嘻嘻][嘻嘻]回来吃

In [182]:
wb_tn_emoji_diff = np.setdiff1d(y_test_wb_1_emoji_metrics[1], y_test_wb_0_emoji_metrics[1])
np.array(X_test_wb_1)[wb_tn_emoji_diff][100:200]

array(['恐龙特级可赛号[泪]',
       '我在这里:#扬州市-广陵区#骑鹤下扬州，首航体验团，午餐在晚清第一名园何园，第二次到扬州，感觉一样好，只是上次已经过了四年。[泪] http:',
       '独在异乡为异客[泪]',
       '我的电子秤到货了。我跟@云妈当家 说，等我减去多余的内30斤肉以后，我就不是奇怪的圆脸尖下颏儿，而是真的瓜子儿脸了。云妈对我是什么脸这事儿好像不是很感兴趣，反而对这30斤肉到底是多少非常好奇：30斤哪，哎哟~这要是30斤猪肉那得用多大个盆才盛的下呀……[衰]@#￥%……&*&……^#￥%……[泪]',
       'RT @蒋宇捷: 招人真难 手握一堆headcount 就是找不到合适的人 [晕]', '可以签名吗[泪][泪] ',
       '马尔代夫版的#穷兄极饿# 又到觅食儿的时候了[抓狂]@穷游网',
       '@YOKA化妆品 收到了~拧不动，还没琢磨明白怎么用。只是保质期居然就到5月9号[晕]', '酒瓮否？[疑问]',
       '亲爱的，你想着忙碌中的我，感动[泪]恩，加油，HOLD住[奥特曼][爱你]抓住晚上的时间，努力睡觉。。',
       '#须叔推荐#周末来扫扫盲：“压轴”是倒数第二的意思！[可怜]――偶觉得偶的世界观颠覆了！身为一个码字人，叔一直以为“压轴”是“最后、重磅”的意思。。。[伤心][?]亲们告诉偶叔不是一个人，５５５～～[挤眼]@韩松落 @徐四丫头 @若宣非宣 #所以不看传统戏就是不能了解传统文化么#',
       '昨夜梦境，群星荟萃，众星云集@shiER奶奶哈五七 @远方的家周雪梅 @陈烨萌 @Mus娜娜 @鬼谷靖天 @密斯特余 @赵可 @远方的小马哥 @远小贱 @远方的家打工仔 扰我清梦，寡不敌众啊，你说说，你们这是要闹哪样啊？！[汗][汗][汗][抓狂][抓狂][抓狂]好吧，我在机场了，一会儿飞~~~[飞机][飞机][飞机][飞机]',
       '我要入教[抓狂][鼓掌]',
       '#长假扎堆游#一行白鹭上青天，车子挤在最中间。借问酒家何处有，又被堵在景区口。犹抱琵琶半遮面，车上没带方便面。天生我材必有用，五个小时就不动[失望]',
       '有朋友约上海，白天较量球技，晚上较量酒剂！白天输球的请喝酒，喝酒输的请打球

In [184]:
wb_tp_emoji_diff_rev = np.setdiff1d(y_test_wb_0_emoji_metrics[0], y_test_wb_1_emoji_metrics[0])
np.array(X_test_wb_1)[wb_tp_emoji_diff_rev][0:100]

array(['刚到还没进家门呢[?][?][?][困][困][困] ', '我热4-1干掉凯子！[怒] ',
       '炫酷的大迪小迪来一发[抓狂]这个绝壁解不了啊[嘻嘻]/@老真真真真真五:我会疯掉！！！ ',
       '[抓狂][抓狂][抓狂][抓狂]我请吃饭！ ', '回复@鹏的莹:[晕][晕] ',
       '这是一个热死没人管的季节[汗][汗][汗][泪] 爬丹霞山，从夹缝中终于爬上山，经过了嘉遁亭，来到了99天梯下山，我完全傻了。吓傻[黑线][奥特曼] 热傻? @秀媛大牌 每次见你都这么瘦[爱你][爱你]  我在这里:http:',
       '[鼓掌][鼓掌][心][鲜花]老磨坊官门口店，好感动的一幕，[泪]幸福满溢的女人。。。浪漫什么时候都不晚，爱她就是要告诉她！[鼓掌][爱心传递]',
       '【蓝双】据说王导一年前就准备来钻墙啦~~真是好事多磨现在才看到您矫捷的身手~~拿着超级大南瓜，王导选择主动出击，不料墙洞坚实有力，等再晃过神来，只听王导一句：“我的瓜呢？”[晕][哈哈]最神奇的是谢映玲，眼看下一秒就要落水，硬是hold到最后誓死不从！@王为念微博 @滴玲玲玲 #正大综艺微直播#',
       '[爱你]亲有机会一定试试榴莲乳酪  上回没给亲排上 真心抱歉[泪] ', '聚会很欢乐但是我喝多了[泪] ',
       '老妹和奶牛?[爱你]淘淘是个胆小鬼，不出来?[泪]',
       '从几百万瞬间到零的不寻常变动，虽然并不能影响《璀璨人生》的全国收视第一！[hold住]但希望这样人工操控、背离观众的收视污染远离我们的电视圈！[怒]也送给“被算计”的喜爱《璀璨》的观众朋友们一个好消息，下周六晚（9.14）璀璨主演登陆大本营！欢迎大家继续支持！[鼓掌]',
       'UFO出没，请注意！[衰] Bone新款耳机收纳器【幽浮吸盘卷线盒】上架啦！轻松收纳耳机，具吸盘、手机立架及收纳卷线三大功能；采环保硅胶，手感细致不掉色，可水洗；独特三角造型携带方便，可轻易收纳 (限使用入耳式耳机)；贴心设计可固定式内层耳机插头构造。[亲亲] 三色可选哦~ http:',
       '好运快来[偷笑][偷笑][抓狂][抓狂][抓狂][嘻嘻][嘻嘻][哈哈][哈哈][花心][花心][鼓掌][鼓掌][鼓掌]',
  

In [183]:
wb_tn_emoji_diff_rev = np.setdiff1d(y_test_wb_0_emoji_metrics[1], y_test_wb_1_emoji_metrics[1])
np.array(X_test_wb_1)[wb_tn_emoji_diff_rev][0:100]

array(['[偷笑]', '我也如此[哈哈]求女神', '[鼓掌][鼓掌][鼓掌][鼓掌] ', '最好买个房车，到哪儿住哪儿[笑哈哈]',
       '[哈哈]', '小道消息，我只告诉你：年底。[嘻嘻] ', '我是回了又回！[哈哈] ', '[哈哈] ', '[围观]',
       '你没有那么性感的胡子[哈哈]', '[哈哈]',
       '回复@忧桑的马小七:[吃惊]真的假的？看来，“雨神”的功力只增不减啊！！估计今天晚上过了十一点半全中国都要下雨啦！！！[哈哈] ',
       '[兔子]',
       '回复@蒙古小烧:为神马让老婆拿皮箱的是正常的？还以为有让老婆拿箱子的是不正常的。难道这就是婚姻中的女人和恋爱中的女人的差别？[哈哈] ',
       '我看像超级打手[哈哈]', '回复@邵雪云golf:不是去????[哈哈] ', '[爱你][威武]',
       '[偷笑][偷笑][偷笑][嘘][嘘][嘘][阴险][阴险][阴险] ',
       '回复@中国航天电源王刚:[哈哈]老王把 @大?儿 说的很是耿耿于怀的 ', '[赞]  ', '[偷笑] ',
       '[哈哈]惭愧惭愧！', '[花心][花心][哈哈][哈哈] ',
       '异性相吸嘛。[偷笑]其实你可以先从你身边的妹子下手，不用太漂亮也行啊，先练习嘛。', '[偷笑][偷笑][偷笑]',
       '欣慰！[鼓掌] ', '右一太绝了！ 一语道破玄机！实在佩服[赞]', '确实，这些都是有时差的记者[嘻嘻] ',
       '造访过上环警署[呵呵] 上环闲逛一时内急 求助警员 他想了好久说 你还是去警署吧 并指出位置 我一进门 所有的警员都停下工作 焦急的询问 出啥事了？我羞涩的回答....[害羞] 出来的时候还顺带研究了一下 招募薪水表[吃惊]高级督察月薪4.8-8万哦 ',
       '#段王爷#我妈竟然在船上刷了@李宗翰 同学的信用卡？真是高端、时尚、霸气的老妈！[威武]福大人有钱，老妈你做的太对了！就是应该这么干！',
       '回复@Mio张创:我的还热者 剩下5个酸菜猪肉 你打个飞的来吃吧 还有地三鲜 [哈哈] ', '笑死了[哈哈][哈哈][哈哈] ',
       '你们俩今天这是要闹哪样？[

## Sketch

In [61]:
weibo_emoji_classifier("小何，你觉得呢？[懒得理你]")

[{'label': 'LABEL_0', 'score': 0.5992363691329956}]

In [62]:
weibo_emoji_classifier("小何，你觉得呢？")

[{'label': 'LABEL_1', 'score': 0.5085890293121338}]

In [63]:
weibo_no_emoji_classifier("小何，你觉得呢？[懒得理你]")

[{'label': 'LABEL_0', 'score': 0.6743822693824768}]

In [64]:
weibo_no_emoji_classifier("小何，你觉得呢？")

[{'label': 'LABEL_1', 'score': 0.970901370048523}]

In [65]:
uer_dianping_classifier("小何，你觉得呢？[懒得理你]")

[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.9601520299911499}]

In [66]:
uer_dianping_classifier("小何，你觉得呢？")

[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.7702454328536987}]

In [33]:
fp_idx[0:5]

array([ 14,  98, 121, 133, 258])

In [34]:
np.array(y_test_wb_1)[fp_idx[0:5]]

array([0, 0, 0, 0, 0])

In [35]:
np.array(X_test_wb_1)[fp_idx][0:5]

array(['这个有人可以效仿一下[偷笑] ', '回复@周涛ufo:#张译##读书会#等着收看[鼓掌] ', '自己发片逗自己玩哦！[偷笑]',
       '[偷笑]', '回复@庄泳joey:谢谢亲爱的[爱你][爱你] '], dtype='<U227')

In [36]:
fp_reviews = np.array(X_test_wb_1)[fp_idx][0:5]

In [37]:
for fp_review in fp_reviews:
    print(weibo_emoji_classifier(fp_review)[0])

{'label': 'LABEL_1', 'score': 0.8836879730224609}
{'label': 'LABEL_1', 'score': 0.9949190020561218}
{'label': 'LABEL_1', 'score': 0.9675623774528503}
{'label': 'LABEL_1', 'score': 0.6853876709938049}
{'label': 'LABEL_1', 'score': 0.9676677584648132}


In [38]:
fn_idx[0:5]

array([ 35,  36, 158, 169, 235])

In [39]:
np.array(y_test_wb_1)[fn_idx[0:5]]

array([1, 1, 1, 1, 1])

In [40]:
np.array(X_test_wb_1)[fn_idx][0:5]

array(['刚到还没进家门呢[?][?][?][困][困][困] ', '老失眠，原来不是真的失眠[嘻嘻]是智商高',
       '哭了~[嘻嘻]@徐小虎哪儿都没有', '[泪][太开心]感动死了，你是第二个祝我生日快乐的，果不负你二媛的称号[鼓掌][鼓掌]',
       '回复@大胃肚夫:来吧，各种美食管够，一定要让您扶墙出去[偷笑] '], dtype='<U227')

In [41]:
fn_reviews = np.array(X_test_wb_1)[fn_idx][0:5]

In [42]:
for fn_review in fn_reviews:
    print(weibo_emoji_classifier(fn_review)[0])

{'label': 'LABEL_0', 'score': 0.9970749616622925}
{'label': 'LABEL_0', 'score': 0.7466278672218323}
{'label': 'LABEL_0', 'score': 0.7240278124809265}
{'label': 'LABEL_0', 'score': 0.7788592576980591}
{'label': 'LABEL_0', 'score': 0.9106667637825012}


Around 5% of data has length less than or equal to 1 when emoji is removed, so they may be ambiguous when classifying. We set a threshold of 10 to filter out short texts, and re-assess.

In [129]:
# remove texts with length shorter than 10
X_test_1_filtered = []
y_test_1_filtered = []
X_test_2_filtered = []
y_test_2_filtered = []

for i in range(len(X_test_2)):
    if len(X_test_2[i]) >= 10:
        X_test_1_filtered.append(X_test_1[i])
        y_test_1_filtered.append(y_test_1[i])
        X_test_2_filtered.append(X_test_2[i])
        y_test_2_filtered.append(y_test_2[i])